## Download the data

In [84]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

class GetDailyCountyData:
    def __init__(
            self,
            start_date,
            end_date,
            base_url,
            states_shp_url,
            state
    ):
        self.date_range = pd.date_range(start_date, end_date)
        self.base_url = base_url
        state_shp = gpd.read_file(states_shp_url)
        self.state_shp = state_shp[state_shp.NAME == state].iloc[0]
    
    def download(self):
        state_specific_df = pd.DataFrame()
        for d in tqdm(self.date_range):
            csv_url = self.base_url.format(d.year,'%02d' % d.month, '%02d' % d.day)
            df = pd.read_csv(csv_url, engine='pyarrow')
            df_o = gpd.GeoDataFrame(df[['lng_o','lat_o']],  geometry=gpd.points_from_xy(df.lng_o, df.lat_o))
            df_d = gpd.GeoDataFrame(df[['lng_d','lat_d']],  geometry=gpd.points_from_xy(df.lng_d, df.lat_d))
            df = df[(self.state_shp.geometry.contains(df_o.geometry)) & (self.state_shp.geometry.contains(df_d.geometry))]
            state_specific_df = pd.concat([state_specific_df, df], axis=0)
        return state_specific_df

In [85]:
# Get data on the kincaid fire
base_url = "https://github.com/GeoDS/COVID19USFlows-DailyFlows/raw/master/daily_flows/county2county/daily_county2county_{}_{}_{}.csv"
states_shp_url = 'https://github.com/joncutrer/geopandas-tutorial/raw/master/data/usa-states-census-2014.shp'

get_daily_data = GetDailyCountyData('2019-07-23','2019-11-06', base_url, states_shp_url, 'California')
kincaid = get_daily_data.download()
kincaid.to_csv('california_aug_2019_nov_2019.csv')

100%|██████████| 2/2 [00:42<00:00, 21.38s/it]


,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date,visitor_flows,pop_flows
19972,6005,6037,-120.651116,38.44639,-118.261017,34.198001,2019-01-01,18,381.0
19973,6005,6071,-120.651116,38.44639,-116.178461,34.841438,2019-01-01,3,63.0
19974,6005,6085,-120.651116,38.44639,-121.696279,37.232484,2019-01-01,8,169.0
19979,6005,6001,-120.651116,38.44639,-121.917885,37.650547,2019-01-01,12,254.0
19980,6005,6007,-120.651116,38.44639,-121.600672,39.666929,2019-01-01,5,105.0
...,...,...,...,...,...,...,...,...,...
581713,6091,6115,-120.515996,39.58040,-121.351263,39.269008,2019-01-02,2,59.0
581714,6091,6057,-120.515996,39.58040,-120.768752,39.301370,2019-01-02,18,538.0
581715,6091,6097,-120.515996,39.58040,-122.922545,38.525293,2019-01-02,1,29.0
581716,6091,6035,-120.515996,39.58040,-120.594333,40.673590,2019-01-02,2,59.0
